In [1]:
%pip install langchain
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ["OPENAI_API_KEY"] = "<OpenAI_APIのトークン>"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.5-py3-none-any.whl size=67620 sha256=2efca9ad8e294f7a1fb80f087fbd34ea35980c0e21e9a0529e98ea710dd14056
  Stored in directory: /root/.cache/pip/wheels/a7/47/99/8273a59fbd59c303e8ff175416d5c1c9c03a2e83ebf7525a99
Successfully built openai


In [4]:
pdf_file_name = list(uploaded.keys())[0]
target_pdf_file_path = '/content/' + pdf_file_name

In [5]:
import os
if not os.path.exists('/content/output'):
    os.mkdir('/content/output')

In [6]:
%cd /content
!wget https://github.com/kermitt2/grobid/archive/0.7.2.zip
!unzip 0.7.2.zip
%cd grobid-0.7.2/

Streaming output truncated to the last 5000 lines.
  inflating: grobid-0.7.2/grobid-trainer/resources/dataset/entities/chemistry/corpus/WO2007038867.HC.class-names.xml  
  inflating: grobid-0.7.2/grobid-trainer/resources/dataset/entities/chemistry/corpus/WO2007038867.HC.ligand-names.xml  
  inflating: grobid-0.7.2/grobid-trainer/resources/dataset/entities/chemistry/corpus/WO2007038867.HC.substance-names.xml  
  inflating: grobid-0.7.2/grobid-trainer/resources/dataset/entities/chemistry/corpus/WO2007038867.composite.xml  
  inflating: grobid-0.7.2/grobid-trainer/resources/dataset/entities/chemistry/corpus/WO2007038867.lines.xml  
  inflating: grobid-0.7.2/grobid-trainer/resources/dataset/entities/chemistry/corpus/WO2007038867.words.xml  
  inflating: grobid-0.7.2/grobid-trainer/resources/dataset/entities/chemistry/corpus/WO2007038869.HC.chemical-names.xml  
  inflating: grobid-0.7.2/grobid-trainer/resources/dataset/entities/chemistry/corpus/WO2007038869.HC.chemical-relations.xml  
  inf

In [7]:
!./gradlew clean install

..........10%...........20%...........30%..........40%...........50%...........60%..........70%...........80%...........90%...........100%

Welcome to Gradle 7.1.1!

Here are the highlights of this release:
 - Faster incremental Java compilation
 - Easier source set configuration in the Kotlin DSL

For more details see https://docs.gradle.org/7.1.1/release-notes.html

Starting a Gradle Daemon (subsequent builds will be faster)


> Starting Daemon> IDLE<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING [1s]> Evaluating settings<-------------> 0% INITIALIZING [2s]> Evaluating settings > Compiling settings file 'settings.gradle'<-------------> 0% INITIALIZING [3s]> Evaluati

In [8]:
!java -Xmx4G -jar grobid-core/build/libs/grobid-core-0.7.2-onejar.jar -gH grobid-home -dIn /content/ -dOut /content/output -exe processFullText

Feb 12, 2023 1:49:36 PM org.grobid.core.main.GrobidHomeFinder getGrobidHomePathOrLoadFromClasspath
Feb 12, 2023 1:49:36 PM org.grobid.core.main.GrobidHomeFinder findGrobidHomeOrFail
Feb 12, 2023 1:49:36 PM org.grobid.core.main.GrobidHomeFinder findGrobidHomeOrFail
Feb 12, 2023 1:49:36 PM org.grobid.core.main.GrobidHomeFinder findGrobidHomeOrFail
Feb 12, 2023 1:49:36 PM org.grobid.core.main.GrobidHomeFinder getGrobidHomePathOrLoadFromClasspath
Feb 12, 2023 1:49:36 PM org.grobid.core.main.GrobidHomeFinder findGrobidHomeOrFail
Feb 12, 2023 1:49:36 PM org.grobid.core.main.GrobidHomeFinder findGrobidHomeOrFail
Feb 12, 2023 1:49:36 PM org.grobid.core.main.GrobidHomeFinder findGrobidHomeOrFail
Feb 12, 2023 1:49:36 PM org.grobid.core.main.GrobidHomeFinder findGrobidConfigOrFail
Processing: /content/radiol.230171 (1).pdf
Feb 12, 2023 1:49:36 PM org.grobid.core.main.LibraryLoader load
INFO: Loading external native sequence labelling library
Feb 12, 2023 1:49:36 PM org.grobid.core.main.LibraryLoa

In [9]:
from xml.etree.ElementTree import Element

def get_text(element: Element) -> str:
    text = ''
    for elem in element.iter():
        if elem.text:
            text += elem.text
        if elem.tail:
            text += elem.tail
    return text

class Section:
    def __init__(self, title: str = "", body: str = ""):
        self.title = title
        self.body = body

In [10]:
import openai
def completion(text):
    openai.api_key = API_KEY
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=text,
        max_tokens=1200,
        temperature=0.7,
        frequency_penalty=1.0,
    )
    return response['choices'][0]['text']

# gpt用のプロンプトに変換
def create_prompt(english_title, english_section_title, english_bodytext):
    return f"""英語の研究論文の一部を日本語で要約するタスクを行います。
これは「{english_title}」というタイトルの論文の「{english_section_title}」というセクションの文章です。
以下のルールに従ってください。

・リスト形式で出力する (先頭は - を使う)
・簡潔に表現する
・不明な単語や人名と思われるものは英語のまま表示する

それでは開始します。

英語の論文の一部:
{english_bodytext}

日本語で要約した文章:"""

In [11]:
import os
import json
import xml.etree.ElementTree as ET

file_name = os.path.splitext(pdf_file_name)[0]
xml_path = f"./output/{file_name}.tei.xml"
tree = ET.parse(xml_path)
root = tree.getroot()

In [12]:
paper_title = root[0][0][0][0].text

In [13]:
sections = []
for div in root[1][0]:
    section = Section("", "")
    for element in div:
        if (element.tag == "{http://www.tei-c.org/ns/1.0}head"):
            section.title = element.text
        if (element.tag == "{http://www.tei-c.org/ns/1.0}p"):
            section.body += get_text(element)
    if section.body != "":
        sections.append(section)

In [14]:
for section in sections:
    print(section.title)
    print("----")
    print(section.body)
    print("~~~~~~~~~~~~~~~")

I n p r e s s
----
This copy is for personal use only. To order printed copies, contact reprints@rsna.org "You cannot endow even the best machine with initiative." -Walter Lippmann In this issue of Radiology, the article by Biswas explains a breakthrough in artificial intelligence (AI) text generation called ChatGPT. This AI-based writing assistant can generate text for medical writing, including scientific articles. What is fascinating, is that the paper is a selfreference. It was almost all written using ChatGPT, except for the introduction, cautions, and edits by the human author. The paper headings reflect the exact prompts inputted by the human author. For example, one header prompt that created the next paragraph written by ChatGPT is "will chat gpt replace human medical writer?" This makes it an article that talks about itself (meta-text). Thus, proving the capabilities of ChatGPT as an assistive tool in medical writing (1).The article explains key concepts like medical writing,

In [15]:
markdown_text = ""
for section in sections:
    prompt = create_prompt(paper_title, section.title, section.body)
    result = completion(prompt)
    markdown_text += "\n\n## " + section.title
    markdown_text += "\n" + result

In [16]:
from IPython.display import display, Markdown, Latex
display(Markdown(markdown_text))



## I n p r e s s

- バスワス氏の論文では、AIテキスト生成技術「ChatGPT」に関する突破が説明されている。
- 論文のほとんどはChatGPTによって書かれたが、導入部分や注意事項、編集などは人間の著者によって行われた。
- ChatGPTを医学的なライティングの支援ツールとして使用する可能性を実証した。
- 医学ライティング、自然言語処理（NLP）、生成的前処理トランスフォーマー（GTP）、ChatGPTなどの重要なコンセプトが説明されている。
- ChatGPTを用いた医学ライティングで使用可能なものとしてはリアルタイムアシスト機能、临床試験プロトコルの生成からレポート作成までの幅広い分野があり、また各国語への翻訳も可能だと述べられている。 
- その結果1,000,000人が5日以内に利用することを契機に世界中の注目を集めた

## I n p r e s s

- GPT（自然言語処理）は、ヘルスケアにおいて有限な制限があると考えられてきたが、ChatGPTにより突破口を作り出した。
- しかしながら、ChatGPTも多くの制限を持っている。倫理的懸念やプラグライズムを回避するための編集作業の要求、商用目的のAI生成コンテンツの利用での法律上の懸念などがある。
- 原創性の欠如も問題であり、ChatGPTは与えられたデータに依存することから重複文章の生成を引き起こす可能性がある。不正確さ、バイアス化、透明性もまた問題となっている。
- AIを利用した文章作成プロセスの中にAIの役割の鮮明さが必要だとOpenAIでは述べている。
- プラグライズムを避け入力された文章の正確性の確認やパラフレーズ化など使う際の配慮事項もあり ChatGPT の効果的利用法は存在することが判明している。
- AI開発会社であるOpenAIは「ChatGPT は実在感のあって不正確または無意味な回答を吐くことも有り得ます」(4) に警告しています 人間の無関心から生じう可能性のある危害に対処す��う注意事項も考慮されなくては何一